In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

In [5]:
df=pd.read_csv('ParisHousingClass.csv')

In [6]:
df['category']=df['category'].map({'Luxury':1,'Basic':0})
cols= df.columns.tolist()
df.head()

,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price,category
0,75523,3,0,1,63,9373,3,8,2005,0,1,4313,9005,956,0,7,7559081.5,0
1,80771,39,1,1,98,39381,8,6,2015,1,0,3653,2436,128,1,2,8085989.5,1
2,55712,58,0,1,19,34457,6,8,2021,0,0,2937,8852,135,1,9,5574642.1,0
3,32316,47,0,0,6,27939,10,4,2012,0,1,659,7141,359,0,3,3232561.2,0
4,70429,19,1,1,90,38045,3,7,1990,1,0,8435,2429,292,1,4,7055052.0,1


In [8]:
train,test,valid=np.split(df.sample(frac=1),[int(0.6*len(df)),int(0.8*len(df))])
train

/opt/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price,category
5933,28605,2,0,1,25,63712,7,6,2003,1,1,1624,1800,492,0,1,2866459.3,0
765,67663,1,1,0,49,17835,2,10,2014,0,1,7856,4653,725,1,2,6775397.4,0
4650,1906,16,0,0,30,9172,1,10,2006,0,1,1576,3923,883,0,7,192160.6,0
619,30572,24,0,0,56,32061,3,9,2008,0,0,171,2208,401,1,2,3058110.8,0
3472,4456,65,1,0,79,14188,2,10,2015,0,1,9586,8645,745,1,2,451747.5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1098,86614,65,0,1,63,15680,10,6,1996,0,1,1472,8750,919,1,1,8671187.6,0
9470,97863,31,1,0,38,11088,2,8,1992,1,1,1887,2659,321,1,4,9794827.2,0
1788,73080,96,1,0,28,17491,3,2,1990,0,1,2800,3779,687,1,2,7311244.0,0
7690,34341,31,0,1,58,60068,1,1,1990,1,0,7361,1398,773,1,6,3438239.0,0


In [9]:
def scale_dataset(dataframe,oversample=False):
    x=dataframe[dataframe.columns[0:-1]].values
    y=dataframe[dataframe.columns[-1]].values
    scalar=StandardScaler()
    x=scalar.fit_transform(x)
    data=np.hstack((x,np.reshape(y,(-1,1))))
    if oversample:
        ros=RandomOverSampler()
        x,y=ros.fit_resample(x,y)
    return data,x,y


In [10]:
train,x_train,y_train=scale_dataset(train,oversample=True)
test,x_test,y_test=scale_dataset(test,oversample=False)
valid,x_valid,y_valid=scale_dataset(valid,oversample=False)

LOGISTIC REGRESSION

In [14]:
from sklearn.linear_model import LogisticRegression
logic_model=LogisticRegression()
logic_model.fit(x_train,y_train)
y_pred=logic_model.predict(x_test)
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1745
           1       1.00      1.00      1.00       255

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000



SVM MODEL

In [15]:
from sklearn.svm import SVC
svm_model=SVC()
svm_model.fit(x_train,y_train)
y_pred=svm_model.predict(x_test)
print(classification_report(y_pred,y_test))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1745
           1       1.00      1.00      1.00       255

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000



In [17]:
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Assuming X_train, X_test, y_train, y_test are already prepared

# Initialize Neural Network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer with linear activation for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(x_train, y_train, epochs=50, batch_size=32, validation_data=(x_test, y_test))

# Evaluate the model
loss = model.evaluate(x_test, y_test)
print(f'Mean Squared Error on test data: {loss}')

# Predictions
y_pred = model.predict(x_test)

from sklearn.metrics import mean_squared_error, r2_score

# Calculate evaluation metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

Epoch 1/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 461us/step - loss: 0.1733 - val_loss: 0.0322
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 323us/step - loss: 0.0417 - val_loss: 0.0126
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 321us/step - loss: 0.0223 - val_loss: 0.0060
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 321us/step - loss: 0.0143 - val_loss: 0.0031
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 323us/step - loss: 0.0100 - val_loss: 0.0028
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 328us/step - loss: 0.0068 - val_loss: 0.0021
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 323us/step - loss: 0.0053 - val_loss: 0.0026
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 321us/step - loss: 0.0037 - val_loss: 0.0021
Epoch 9/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 321us/step - loss: 0.0030 - val_loss: 0.0024
Epoch 10/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 319us/step - loss: 0.0025 - val_loss: 0.0018
Epoch 11/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 316us/step - loss: 0.0018 - val_loss: 0.0018
Epoch 12/50
328/328 ━━━━━━━━━━